# MC ε-贪心算法教程：更实用的蒙特卡洛方法

## 📖 为什么需要 MC ε-贪心？

在 `MC Basic` 算法中，我们学习了蒙特卡洛方法的基本原理。但它有一个很强的假设：**探索性起点 (Exploring Starts)**。这意味着为了评估所有状态-动作对，我们需要从每一个可能的 (s,a) 对开始生成回合。

在现实世界中，这个假设往往难以满足。例如，在机器人控制任务中，我们可能无法随意设置机器人的初始状态和初始动作。

**MC ε-贪心算法**通过引入“探索”机制解决了这个问题，使其更加实用：

- **无需探索性起点**：通过 ε-贪心策略在学习过程中进行探索。
- **平衡探索与利用**：以 1-ε 的概率选择当前最优动作（利用），以 ε 的概率随机选择一个动作（探索）。
- **在线策略 (On-policy)**：用于生成数据的策略和被评估改进的策略是同一个。
- **保证收敛**：在一定条件下，它能收敛到最优策略。

## 🎯 本教程目标

我们将从零开始实现 MC ε-贪心算法，并在同一个 4×4 网格世界中进行可视化：

- **核心内容**：
  - 理解 ε-贪心策略如何平衡探索与利用。
  - 实现基于回合采样的 on-policy 蒙特卡洛控制。
  - 可视化 Q 值和策略在探索中的收敛过程。

- **学习路径**：
  1. 理解 ε-贪心策略。
  2. 构建支持随机起始回合的环境。
  3. 实现 MC ε-贪心核心算法。
  4. 可视化学习过程（GIF 动画）。
  5. 分析与 MC Basic 的异同。

---

让我们开始探索这种更强大的无模型学习方法！

## 第一步：安装和导入必要的库

In [ ]:
# 导入必要的库
import gymnasium as gym
from gymnasium import spaces
import numpy as np
from typing import Optional, Tuple, Dict, Any, List, Annotated
from collections import defaultdict

# 设置 matplotlib 后端（必须在导入 pyplot 之前）
import matplotlib
matplotlib.use('Agg')  # 使用非交互式后端，适合云端环境

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.colors import LinearSegmentedColormap
from IPython.display import display, clear_output, Image as IPImage
import imageio
from io import BytesIO

# 配置 matplotlib 中文显示
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans', 'Arial Unicode MS', 'WenQuanYi Micro Hei']
plt.rcParams['axes.unicode_minus'] = False

print(f"✅ Gymnasium 版本: {gym.__version__}")
print(f"✅ NumPy 版本: {np.__version__}")
print(f"✅ Matplotlib 版本: {matplotlib.__version__}")
print(f"✅ Matplotlib 后端: {matplotlib.get_backend()}")
print(f"✅ 中文字体配置完成")
print(f"✅ imageio 已导入，支持 GIF 动画生成")

## 第二步：创建支持 MC 学习的网格世界环境

为了适应 ε-贪心算法，我们需要对环境做一些微调。主要区别在于回合的生成方式：

- **随机起点**：回合可以从任意非终止状态随机开始。
- **策略驱动**：整个回合都由一个给定的策略（即 ε-贪心策略）驱动。

In [ ]:
class GridWorldEpsilonGreedy(gym.Env):
    """支持 MC ε-贪心学习的网格世界环境"""
    
    def __init__(
        self,
        size: Annotated[int, "网格的边长"] = 4,
        rewards: Annotated[Optional[np.ndarray], "可选的奖励矩阵"] = None,
    ):
        super().__init__()
        self.size = size
        
        # 设置奖励矩阵
        if rewards is None:
            self.rewards = np.array([
                [0, 0, -1, -1],
                [0, -1, -1, 1],
                [0, -1, 0, 0],
                [-1, 0, 0, -1]
            ])
        else:
            self.rewards = rewards
        
        # 目标位置和障碍物位置
        self.target_pos = np.argwhere(self.rewards == 1)[0]
        self.obstacle_mask = (self.rewards == -1)
        
        # 动作空间：0=上, 1=下, 2=左, 3=右
        self.action_space = spaces.Discrete(4)
        self.observation_space = spaces.Discrete(size * size)
        
        # 动作映射
        self._action_to_direction = {
            0: np.array([-1, 0]),  # 上
            1: np.array([1, 0]),   # 下
            2: np.array([0, -1]),  # 左
            3: np.array([0, 1]),   # 右
        }
        
        self._agent_location = np.array([0, 0])
    
    def _pos_to_state(
        self,
        pos: Annotated[np.ndarray, "二维位置坐标"],
    ) -> Annotated[int, "对应的一维状态索引"]:
        """将二维位置转换为一维状态索引"""
        return pos[0] * self.size + pos[1]
    
    def _state_to_pos(
        self,
        state: Annotated[int, "一维状态索引"],
    ) -> Annotated[np.ndarray, "对应的二维位置坐标"]:
        """将一维状态索引转换为二维位置"""
        return np.array([state // self.size, state % self.size])
    
    def reset(
        self,
        start_pos: Annotated[Optional[np.ndarray], "可选的起始位置坐标"] = None,
    ) -> Annotated[Tuple[int, Dict[str, Any]], "初始状态索引及附加信息字典"]:
        """重置环境，可以指定或随机选择起始位置"""
        if start_pos is not None:
            self._agent_location = np.array(start_pos)
        else:
            # 随机选择一个非终止、非障碍的起始位置
            while True:
                pos = np.random.randint(0, self.size, size=2)
                if not np.array_equal(pos, self.target_pos) and not self.obstacle_mask[pos[0], pos[1]]:
                    self._agent_location = pos
                    break
        
        state = self._pos_to_state(self._agent_location)
        return state, {}
    
    def step(
        self,
        action: Annotated[int, "要执行的动作编号"],
    ) -> Annotated[Tuple[int, float, bool, bool, Dict[str, Any]], "新状态、奖励、终止标志、截断标志及额外信息"]:
        """执行动作"""
        direction = self._action_to_direction[action]
        new_location = self._agent_location + direction
        
        if (0 <= new_location[0] < self.size and 0 <= new_location[1] < self.size):
            self._agent_location = new_location
        
        reward = self.rewards[self._agent_location[0], self._agent_location[1]]
        terminated = np.array_equal(self._agent_location, self.target_pos)
        state = self._pos_to_state(self._agent_location)
        return state, reward, terminated, False, {}
    
    def generate_episode(
        self,
        policy: Annotated[np.ndarray, "策略概率矩阵"],
        max_steps: Annotated[int, "单个回合的最大步数"] = 100,
    ) -> Annotated[List[Tuple[int, int, float]], "状态-动作-奖励序列"]:
        """从随机状态开始，根据策略生成一个完整的回合"""
        state, _ = self.reset() # 随机起点
        episode = []
        
        for _ in range(max_steps):
            action = np.random.choice(self.action_space.n, p=policy[state])
            next_state, reward, terminated, _, _ = self.step(action)
            episode.append((state, action, reward))
            if terminated:
                break
            state = next_state
        
        return episode

# 创建环境实例
env = GridWorldEpsilonGreedy(size=4)
print(f"✅ GridWorldEpsilonGreedy 环境创建完成")
print(f"   - 网格大小: {env.size}×{env.size}")
print(f"   - 动作空间: {env.action_space} (0=上, 1=下, 2=左, 3=右)")
print(f"   - 目标位置: {env.target_pos}")
print(f"\n   奖励矩阵:")
print(env.rewards)

## 第三步：理解 MC ε-贪心算法

### 🧮 ε-贪心策略

为了确保在学习过程中有足够的探索，我们使用 **ε-贪心策略**。对于任意状态 $s$，策略 $\pi$ 定义如下：

$$
\pi(a|s) = 
\begin{cases} 
1 - \epsilon + \frac{\epsilon}{|\mathcal{A}(s)|} & \text{if } a = \arg\max_{a'} Q(s,a') \\ 
\frac{\epsilon}{|\mathcal{A}(s)|} & \text{if } a \neq \arg\max_{a'} Q(s,a') 
\end{cases}
$$

其中：
- $1 - \epsilon$ 是“利用”的概率，即选择当前估计的最好动作。
- $\epsilon$ 是“探索”的概率，即随机选择一个动作。
- $|\mathcal{A}(s)|$ 是状态 $s$ 下的动作数量。

### 🔄 On-Policy MC Control 算法流程

**初始化**：
- 随机初始化 $Q(s,a)$。
- 初始化 `returns(s,a)` 为空列表。
- 根据初始 Q 值创建一个 ε-贪心策略 $\pi$。

**循环（直到收敛或达到最大回合数）：**

1. **生成回合**：使用当前策略 $\pi$ 从一个随机起点生成一个完整的回合。
   - `(S_0, A_0, R_1), (S_1, A_1, R_2), ..., (S_{T-1}, A_{T-1}, R_T)`

2. **计算回报**：对于回合中每个时间步 $t=0, 1, ..., T-1$：
   - 计算从该步开始的折扣回报 $G_t = R_{t+1} + \gamma R_{t+2} + ...$

3. **更新 Q 值**：对于回合中出现的每一个状态-动作对 $(S_t, A_t)$：
   - 将回报 $G_t$ 添加到 `returns(S_t, A_t)` 列表中。
   - 更新 $Q(S_t, A_t)$ 为 `returns(S_t, A_t)` 的平均值。

4. **更新策略**：根据更新后的 Q 值，改进 ε-贪心策略 $\pi$。

## 第四步：定义算法参数

In [3]:
# --- 算法参数 ---
GRID_SIZE = 4           # 网格大小
GAMMA = 0.9             # 折扣因子
EPSILON = 0.1           # ε-贪心策略中的 ε 值
NUM_EPISODES = 5000     # 总共要运行的回合数
MAX_STEPS = 100         # 每个回合的最大步数

# 动作名称（用于可视化）
ACTION_NAMES = ['↑', '↓', '←', '→']

print(f"✅ 算法参数配置完成")
print(f"   - 网格大小: {GRID_SIZE}×{GRID_SIZE}")
print(f"   - 折扣因子 γ: {GAMMA}")
print(f"   - Epsilon ε: {EPSILON}")
print(f"   - 总回合数: {NUM_EPISODES}")
print(f"   - 动作空间: {ACTION_NAMES}")

✅ 算法参数配置完成
   - 网格大小: 4×4
   - 折扣因子 γ: 0.9
   - Epsilon ε: 0.1
   - 总回合数: 5000
   - 动作空间: ['↑', '↓', '←', '→']


## 第五步：实现 MC ε-贪心核心算法

In [ ]:
def create_epsilon_greedy_policy(
    Q: Annotated[np.ndarray, "状态-动作价值表"],
    epsilon: Annotated[float, "ε-贪心策略中的探索概率"],
    num_actions: Annotated[int, "动作空间的大小"],
) -> Annotated[np.ndarray, "ε-贪心策略概率矩阵"]:
    """根据 Q 值创建一个 ε-贪心策略"""
    num_states = Q.shape[0]
    policy = np.ones((num_states, num_actions)) * epsilon / num_actions
    
    for state in range(num_states):
        best_action = np.argmax(Q[state])
        policy[state, best_action] += (1.0 - epsilon)
    
    return policy

def get_policy_matrix(
    policy: Annotated[np.ndarray, "策略概率矩阵"],
    grid_size: Annotated[int, "网格的边长"],
) -> Annotated[np.ndarray, "用于可视化的策略箭头矩阵"]:
    """将策略转换为箭头矩阵用于可视化"""
    policy_arrows = np.empty((grid_size, grid_size), dtype=object)
    
    for state in range(grid_size * grid_size):
        row = state // grid_size
        col = state % grid_size
        best_action = np.argmax(policy[state])
        policy_arrows[row, col] = ACTION_NAMES[best_action]
    
    return policy_arrows

print("✅ MC ε-贪心核心函数定义完成")

## 第六步：实现可视化函数

可视化函数与 `MC Basic` 基本相同，但标题和迭代计数方式有所调整，以反映是基于回合数进行更新。

In [ ]:
def render_mc_epsilon_greedy_to_array(
    rewards: Annotated[np.ndarray, "奖励矩阵"],
    Q: Annotated[np.ndarray, "状态-动作价值表"],
    policy_arrows: Annotated[np.ndarray, "策略箭头矩阵"],
    episode_num: Annotated[int, "当前回合数"],
) -> Annotated[np.ndarray, "用于生成动画的 RGB 数组"]:
    """
    将当前 MC ε-贪心状态渲染为 RGB 数组（用于 GIF 生成）
    """
    fig, axes = plt.subplots(1, 3, figsize=(18, 6), dpi=80)
    
    # 配色方案
    reward_cmap = LinearSegmentedColormap.from_list('reward', ['red', 'white', 'green'])
    value_cmap = 'viridis'
    
    # 1. 绘制奖励矩阵
    ax1 = axes[0]
    im1 = ax1.imshow(rewards, cmap=reward_cmap, vmin=-1, vmax=1)
    ax1.set_title(f'Reward Matrix', fontsize=14, fontweight='bold')
    ax1.set_xticks(range(GRID_SIZE))
    ax1.set_yticks(range(GRID_SIZE))
    
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            ax1.text(j, i, f'{rewards[i, j]:.0f}',
                    ha="center", va="center", color="black", fontsize=12)
    
    plt.colorbar(im1, ax=ax1, fraction=0.046)
    
    # 2. 绘制 Q 值矩阵（显示最大 Q 值）
    ax2 = axes[1]
    Q_max = Q.reshape(GRID_SIZE, GRID_SIZE, 4).max(axis=2)
    im2 = ax2.imshow(Q_max, cmap=value_cmap)
    ax2.set_title(f'Q-Value Matrix (max) - Episode {episode_num}', fontsize=14, fontweight='bold')
    ax2.set_xticks(range(GRID_SIZE))
    ax2.set_yticks(range(GRID_SIZE))
    
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            ax2.text(j, i, f'{Q_max[i, j]:.2f}',
                    ha="center", va="center", 
                    color="white" if Q_max[i, j] < (Q_max.max()/2 if Q_max.max() > 0 else 0.5) else "black",
                    fontsize=10)
    
    plt.colorbar(im2, ax=ax2, fraction=0.046)
    
    # 3. 绘制策略矩阵
    ax3 = axes[2]
    policy_display = np.zeros((GRID_SIZE, GRID_SIZE))
    im3 = ax3.imshow(policy_display, cmap='gray', vmin=0, vmax=1, alpha=0.1)
    ax3.set_title(f'Policy Matrix - Episode {episode_num}', fontsize=14, fontweight='bold')
    ax3.set_xticks(range(GRID_SIZE))
    ax3.set_yticks(range(GRID_SIZE))
    
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            if policy_arrows[i, j]:
                ax3.text(j, i, policy_arrows[i, j],
                        ha="center", va="center", 
                        color="blue", fontsize=24, fontweight='bold')
    
    # 整体标题
    fig.suptitle(f'MC ε-Greedy Algorithm: Episode {episode_num}', 
                 fontsize=16, fontweight='bold', y=0.98)
    
    plt.tight_layout()
    
    # 转换为 RGB 数组
    fig.canvas.draw()
    buf = fig.canvas.buffer_rgba()
    rgb_array = np.asarray(buf)
    rgb_array = rgb_array[:, :, :3]  # RGBA to RGB
    plt.close(fig)
    
    return rgb_array

print("✅ 可视化函数定义完成")

## 第七步：执行 MC ε-贪心算法并生成 GIF 动画

In [ ]:
def run_mc_epsilon_greedy(
    gif_filename: Annotated[str, "输出 GIF 文件名"] = 'mc_epsilon_greedy.gif',
    fps: Annotated[int, "生成动画的帧率"] = 10,
) -> Annotated[Tuple[np.ndarray, np.ndarray], "最终 Q 值和策略矩阵"]:
    """
    执行 MC ε-贪心算法并生成 GIF 动画
    """
    print("=" * 70)
    print("🚀 MC ε-贪心算法启动".center(70))
    print("=" * 70)
    
    # 初始化
    num_states = GRID_SIZE * GRID_SIZE
    num_actions = 4
    
    Q = np.zeros((num_states, num_actions))
    returns = defaultdict(list)
    
    frames = []
    frame_interval = NUM_EPISODES // 100  # 每隔多少回合保存一帧
    
    # MC ε-贪心主循环
    for i in range(1, NUM_EPISODES + 1):
        # 1. 创建/更新 ε-贪心策略
        policy = create_epsilon_greedy_policy(Q, EPSILON, num_actions)
        
        # 2. 生成一个回合
        episode = env.generate_episode(policy, MAX_STEPS)
        
        # 3. 计算回报并更新 Q 值
        G = 0
        visited_sa_pairs = set()
        
        # 从后向前遍历回合
        for state, action, reward in reversed(episode):
            G = reward + GAMMA * G
            sa_pair = (state, action)
            
            # 首次访问 (First-visit) MC
            if sa_pair not in visited_sa_pairs:
                returns[sa_pair].append(G)
                Q[state, action] = np.mean(returns[sa_pair])
                visited_sa_pairs.add(sa_pair)
        
        # 4. 定期生成并保存帧
        if i % frame_interval == 0 or i == 1:
            policy_arrows = get_policy_matrix(policy, GRID_SIZE)
            print(f"生成第 {i} 回合的帧...")
            frames.append(render_mc_epsilon_greedy_to_array(env.rewards, Q, policy_arrows, i))
    
    print(f"\n{'=' * 70}")
    print("🎉 MC ε-贪心算法完成！".center(70))
    print(f"{'=' * 70}")
    print(f"✅ 总计回合数: {NUM_EPISODES}")
    
    # 在结尾多添加几帧以便观察最终结果
    policy_arrows = get_policy_matrix(create_epsilon_greedy_policy(Q, 0, num_actions), GRID_SIZE) # 最终贪婪策略
    for _ in range(10):
        frames.append(render_mc_epsilon_greedy_to_array(env.rewards, Q, policy_arrows, NUM_EPISODES))
    
    # 保存 GIF
    print(f"\n正在保存 GIF（共 {len(frames)} 帧）...")
    imageio.mimsave(gif_filename, frames, fps=fps, loop=0)
    
    print(f"\n✅ GIF 动画已保存: {gif_filename}")
    print(f"   - 总帧数: {len(frames)}")
    print(f"   - 帧率: {fps} fps")
    print(f"   - 总回合数: {NUM_EPISODES}")
    
    return Q, policy

# 执行算法
Q_final, policy_final = run_mc_epsilon_greedy(
    gif_filename='mc_epsilon_greedy.gif',
    fps=20
)

# 显示 GIF
print(f"\n正在显示 GIF...")
display(IPImage(filename='mc_epsilon_greedy.gif'))

# 📐 算法伪代码与实现对照

## 🔍 理论算法（图19：基于ε-贪心探索的MC方法）

下面的伪代码展示了教材中描述的算法结构：

```
算法：On-policy MC Control (基于ε-贪心策略)

参数：
  - ε: 探索概率（小的正数）

初始化：
  - 对所有 s ∈ S, a ∈ A(s)：
    * Q(s,a) ← 任意值
    * Returns(s,a) ← 空列表
  - π ← 相对于Q的ε-贪心策略
    
循环（对每个回合）：
  (a) 使用策略π生成一个回合：S₀,A₀,R₁,S₁,A₁,R₂,...,S_T-1,A_T-1,R_T
  (b) G ← 0
  (c) 循环（对回合中每个时间步，从T-1到0）：
      * G ← γG + R_{t+1}【折扣回报递推计算】
      * 除非 (S_t, A_t) 出现在 S₀,A₀,...,S_{t-1},A_{t-1} 中：【First-visit检查】
        - 将 G 追加到 Returns(S_t, A_t)
        - Q(S_t, A_t) ← average(Returns(S_t, A_t))
        - A* ← argmax_a Q(S_t, a)【找到最佳动作】
        - 对所有 a ∈ A(S_t)：【更新ε-贪心策略】
          如果 a = A*:
            π(a|S_t) ← 1 - ε + ε/|A(S_t)|
          否则:
            π(a|S_t) ← ε/|A(S_t)|
```

---

## 💻 本笔记的实现对照

### 📦 **初始化部分** → `run_mc_epsilon_greedy()` 函数开头

**伪代码**：
```
Q(s,a) ← 任意值
Returns(s,a) ← 空列表
π ← 相对于Q的ε-贪心策略
```

**实现代码**：
```python
# run_mc_epsilon_greedy() 函数中：
Q = np.zeros((num_states, num_actions))    # Q(s,a)初始化为0
returns = defaultdict(list)                # Returns(s,a)空列表（动态创建）
# π策略在每个回合开始时创建
```

---

### 🔄 **主循环** → `run_mc_epsilon_greedy()` 中的 for 循环

**伪代码**：
```
循环（对每个回合）
```

**实现代码**：
```python
for i in range(1, NUM_EPISODES + 1):
    # 步骤(a): 生成回合
    policy = create_epsilon_greedy_policy(Q, EPSILON, num_actions)
    episode = env.generate_episode(policy, MAX_STEPS)
    
    # 步骤(b-c): 更新Q值和策略
    # ...（见下文详细对应）
```

> 💡 **关键区别**：与MC Basic不同，这里**每个回合**都更新Q值，而不是迭代所有(s,a)对。

---

### 🎯 **步骤(a)：生成回合** → `create_epsilon_greedy_policy()` + `env.generate_episode()`

**伪代码**：
```
使用策略π生成一个回合：S₀,A₀,R₁,...,S_T
```

**实现代码**：

#### 第1步：创建ε-贪心策略

```python
def create_epsilon_greedy_policy(Q, epsilon, num_actions):
    num_states = Q.shape[0]
    # 初始化所有动作的基础概率为 ε/|A|
    policy = np.ones((num_states, num_actions)) * epsilon / num_actions
    
    for state in range(num_states):
        best_action = np.argmax(Q[state])       # A* = argmax_a Q(s,a)
        policy[state, best_action] += (1.0 - epsilon)  # π(A*|s) = 1-ε+ε/|A|
    
    return policy
```

> 🔑 **ε-贪心策略公式的实现**：
> - 所有动作先设为 `ε/|A|`（探索部分）
> - 最佳动作再加上 `1-ε`（利用部分）
> - 结果：最佳动作概率 = `1-ε+ε/|A|`，其他动作 = `ε/|A|`

#### 第2步：生成回合

```python
# GridWorldEpsilonGreedy.generate_episode() 方法：
def generate_episode(self, policy, max_steps):
    state, _ = self.reset()  # 🔑 随机起点（非探索性起点）
    episode = []
    
    for _ in range(max_steps):
        # 根据ε-贪心策略选择动作
        action = np.random.choice(self.action_space.n, p=policy[state])
        next_state, reward, terminated, _, _ = self.step(action)
        episode.append((state, action, reward))
        if terminated:
            break
        state = next_state
    
    return episode
```

> ⚡ **与MC Basic的核心区别**：
> - ❌ **MC Basic**：从指定的(s,a)开始（探索性起点）
> - ✅ **MC ε-贪心**：从随机状态开始，整个回合由ε-贪心策略驱动

---

### 🧮 **步骤(b-c)：计算回报并更新Q值** → 主循环中的代码

**伪代码**：
```
G ← 0
循环（从T-1到0）：
  G ← γG + R_{t+1}
  除非 (S_t, A_t) 已出现：
    追加 G 到 Returns(S_t, A_t)
    Q(S_t, A_t) ← average(Returns(S_t, A_t))
```

**实现代码**：
```python
# 在 run_mc_epsilon_greedy() 的主循环中：
G = 0
visited_sa_pairs = set()  # 跟踪已访问的(s,a)对【First-visit】

# 从后向前遍历回合
for state, action, reward in reversed(episode):
    G = reward + GAMMA * G  # 折扣回报递推计算
    sa_pair = (state, action)
    
    # First-visit MC：只对首次出现的(s,a)更新
    if sa_pair not in visited_sa_pairs:
        returns[sa_pair].append(G)                  # 追加回报
        Q[state, action] = np.mean(returns[sa_pair]) # 更新Q值为平均
        visited_sa_pairs.add(sa_pair)               # 标记已访问
```

> 📊 **First-visit MC的实现**：
> - 使用 `visited_sa_pairs` 集合跟踪已更新的(s,a)对
> - 只对**首次出现**的(s,a)进行更新
> - 避免同一个(s,a)在一个回合中被多次计数

---

### 🔄 **步骤(c)：策略更新** → 在主循环开始时自动完成

**伪代码**：
```
A* ← argmax_a Q(S_t, a)
对所有 a ∈ A(S_t)：
  如果 a = A*: π(a|S_t) ← 1 - ε + ε/|A|
  否则: π(a|S_t) ← ε/|A|
```

**实现代码**：
```python
# 在每个回合开始时：
policy = create_epsilon_greedy_policy(Q, EPSILON, num_actions)
```

> 💡 **策略更新的隐式实现**：
> - 不需要显式地在回合结束后更新策略
> - 因为 `create_epsilon_greedy_policy()` 总是基于**最新的Q值**创建策略
> - 每个回合开始时重新创建策略，自动反映Q值的变化

---

## 🔑 关键理解点

### 1️⃣ **ε-贪心策略如何消除探索性起点需求？**

**MC Basic的问题**：
- 需要从每个(s,a)开始生成回合（探索性起点）
- 实际应用中往往无法控制起点

**MC ε-贪心的解决方案**：
```python
# 策略本身就包含探索
policy[state, best_action] = 1 - ε + ε/num_actions  # 利用（主要）
policy[state, other_action] = ε/num_actions         # 探索（次要）
```

- ✅ 即使从任意起点开始，ε-贪心策略也会以小概率尝试所有动作
- ✅ 长期来看，所有(s,a)对都会被访问到
- ✅ 无需人为控制起点

### 2️⃣ **On-Policy 学习的特点**

**定义**：用于生成数据的策略 = 被评估和改进的策略

**实现体现**：
```python
# 同一个ε-贪心策略π既用于：
policy = create_epsilon_greedy_policy(Q, EPSILON, num_actions)  # 1. 生成经验
episode = env.generate_episode(policy, MAX_STEPS)               # 2. 评估改进
```

**与MC Basic对比**：
- MC Basic：评估当前策略，改进为贪婪策略（可以看作off-policy）
- MC ε-贪心：始终评估和改进同一个ε-贪心策略（on-policy）

### 3️⃣ **为什么需要大量回合？**

```python
NUM_EPISODES = 5000  # MC ε-贪心需要5000个回合
```

**原因分析**：
1. **随机起点**：每个回合只覆盖一部分(s,a)对
2. **探索概率小**：ε=0.1意味着90%时间在利用，只有10%在探索
3. **收敛需要**：每个(s,a)需要足够多的样本才能准确估计Q值

**与MC Basic对比**：
```python
# MC Basic只需要20次迭代，但每次迭代：
for state in range(16):        # 16个状态
    for action in range(4):    # 4个动作
        for _ in range(10):    # 每个(s,a)采样10次
# 总计：20 × 16 × 4 × 10 = 12800 个回合！
```

---

## 🆚 MC Basic vs MC ε-贪心对比

| 维度 | MC Basic（图18） | MC ε-贪心（图19） |
|------|-----------------|-------------------|
| **探索机制** | 探索性起点（遍历所有(s,a)） | ε-贪心策略（内置探索） |
| **回合起点** | 指定的(s,a)对 | 随机状态 |
| **策略类型** | 确定性贪婪策略 | 随机ε-贪心策略 |
| **学习方式** | Off-policy（评估vs改进策略不同） | On-policy（同一策略） |
| **实际可行性** | 需要控制环境起点（不实用） | 无需控制起点（实用） |
| **收敛保证** | 探索性起点假设下保证 | ε-soft策略下保证 |
| **主循环结构** | 外层：迭代；内层：所有(s,a) | 单层：回合数 |

---

## 🎓 算法精髓总结

### ε-贪心的数学美感

$$
\pi(a|s) = 
\begin{cases} 
1 - \epsilon + \frac{\epsilon}{|A|} & \text{if } a = \arg\max Q(s,a) \\ 
\frac{\epsilon}{|A|} & \text{otherwise}
\end{cases}
$$

**代码实现**：
```python
policy = np.ones((num_states, num_actions)) * epsilon / num_actions  # 基础探索
policy[state, best_action] += (1.0 - epsilon)                        # 额外利用
```

**直觉理解**：
- 所有动作先分享 `ε` 的总概率（公平探索）
- 最佳动作再额外获得 `1-ε` 的概率（重点利用）
- 结果：探索与利用的完美平衡

### First-visit MC的必要性

```python
if sa_pair not in visited_sa_pairs:  # 为什么需要这个检查？
```

**原因**：
- 一个回合中，同一个(s,a)可能出现多次（特别是在有循环的环境中）
- 如果每次出现都更新，会导致样本不独立
- First-visit确保每个回合对每个(s,a)只贡献一个样本

---

## ❓ 思考题

现在你理解了MC ε-贪心的完整实现，思考以下问题：

1. **ε的选择**：如果ε=0会怎样？如果ε=1呢？
   - 提示：思考探索与利用的极端情况

2. **衰减ε**：能否在学习过程中逐渐减小ε？这样做的优缺点是什么？
   - 提示：早期需要多探索，后期需要多利用

3. **与MC Basic的效率对比**：哪个方法在实际中更高效？
   - 提示：考虑环境约束和采样成本

4. **收敛性**：为什么ε-贪心策略能保证找到最优策略？
   - 提示：GLIE（Greedy in the Limit with Infinite Exploration）条件

---

⏸️ **恭喜！你现在完全理解了MC ε-贪心算法的理论与实现。接下来可以学习更高效的时序差分（TD）学习方法！** 🎉

# 🔢 Q值更新的两种等价方法

## ⚠️ 重要说明：图片算法 vs 本笔记实现

你可能注意到，**图片中的算法使用了不同的Q值更新方式**。让我们详细对比这两种方法。

---

## 📊 图片中的算法：增量更新（Incremental Update）

### 伪代码
```
初始化：
  Q(s,a) ← 0，对所有 s,a
  N(s,a) ← 0，对所有 s,a  # 访问次数计数器
  ε ← 1, k ← 1
  π_k ← ε-贪心(Q)

循环（对每个回合 k）：
  对第k个回合采样：S₀,A₀,R₁,...,S_T
  对回合中每个时间步 t：
    计算 G_t ← R_{t+1} + γR_{t+2} + ...
    N(S_t, A_t) ← N(S_t, A_t) + 1           # 增加访问次数
    Q(S_t, A_t) ← Q(S_t, A_t) + (1/N(S_t, A_t)) * (G_t - Q(S_t, A_t))  # 增量更新
  k ← k + 1
  ε ← 1/k  # ε衰减
  π_k ← ε-贪心(Q)
```

### 🔑 关键要素

#### 1. **N(s,a)：访问次数计数器**
```python
N = np.zeros((num_states, num_actions))  # 初始化计数器

# 每次更新时：
N[state, action] += 1  # 增加访问次数
```

**作用**：跟踪每个(s,a)对被访问了多少次，用于计算增量更新的步长。

#### 2. **增量更新公式**
$$
Q(S_t, A_t) \leftarrow Q(S_t, A_t) + \frac{1}{N(S_t, A_t)} \cdot (G_t - Q(S_t, A_t))
$$

**公式解析**：
- `Q(S_t, A_t)`：当前Q值估计
- `G_t`：新观测到的回报
- `G_t - Q(S_t, A_t)`：**TD误差**（新样本与当前估计的差异）
- `1/N(S_t, A_t)`：**学习率**（步长），随着样本增多而减小

**直觉理解**：
```
新估计 = 旧估计 + 学习率 × (新样本 - 旧估计)
       = 旧估计 + 学习率 × 预测误差
```

如果新样本 > 旧估计，Q值增加；如果新样本 < 旧估计，Q值减小。

---

## 💻 本笔记的实现：批量平均（Batch Average）

### 实现代码

```python
# 初始化
returns = defaultdict(list)  # 存储所有回报样本

# 在主循环中：
for state, action, reward in reversed(episode):
    G = reward + GAMMA * G
    sa_pair = (state, action)
    
    if sa_pair not in visited_sa_pairs:
        returns[sa_pair].append(G)                  # 📝 存储回报
        Q[state, action] = np.mean(returns[sa_pair]) # 🧮 计算平均
        visited_sa_pairs.add(sa_pair)
```

### 🔑 关键特点

1. **存储所有样本**：`returns[(s,a)]` 列表保存所有历史回报
2. **重新计算平均**：每次都用 `np.mean()` 计算所有样本的平均值

---

## 🎯 数学等价性证明

### 定理：增量更新 ≡ 批量平均

**设前n个样本的平均值为 $Q_n$**，当收到第 $n+1$ 个新样本 $G_{n+1}$ 时：

#### 批量平均方法：
$$
Q_{n+1} = \frac{1}{n+1} \sum_{i=1}^{n+1} G_i
$$

#### 增量更新方法：
从批量平均公式推导：

$$
\begin{align}
Q_{n+1} &= \frac{1}{n+1} \sum_{i=1}^{n+1} G_i \\
&= \frac{1}{n+1} \left( \sum_{i=1}^{n} G_i + G_{n+1} \right) \\
&= \frac{1}{n+1} \left( n \cdot Q_n + G_{n+1} \right) \\
&= \frac{n}{n+1} Q_n + \frac{1}{n+1} G_{n+1} \\
&= Q_n + \frac{1}{n+1} (G_{n+1} - Q_n) \quad \checkmark
\end{align}
$$

**结论**：两种方法在数学上**完全等价**！

---

## 🔍 具体例子：数值验证

### 场景设置
假设状态-动作对 $(s_0, a_0)$ 在3个回合中的回报分别为：
- 第1个回合：$G_1 = 5.0$
- 第2个回合：$G_2 = 8.0$  
- 第3个回合：$G_3 = 6.0$

### 方法1：批量平均

```python
returns = [5.0, 8.0, 6.0]
Q = np.mean(returns) = (5.0 + 8.0 + 6.0) / 3 = 6.333...
```

### 方法2：增量更新

```python
# 初始化
Q = 0.0
N = 0

# 第1个样本 G₁ = 5.0
N = 1
Q = Q + (1/N) * (5.0 - Q)
  = 0.0 + 1.0 * (5.0 - 0.0)
  = 5.0

# 第2个样本 G₂ = 8.0
N = 2
Q = Q + (1/N) * (8.0 - Q)
  = 5.0 + 0.5 * (8.0 - 5.0)
  = 5.0 + 1.5
  = 6.5

# 第3个样本 G₃ = 6.0
N = 3
Q = Q + (1/N) * (6.0 - Q)
  = 6.5 + (1/3) * (6.0 - 6.5)
  = 6.5 - 0.1666...
  = 6.333...  ✅ 相同！
```

**验证成功**：两种方法得到相同结果 `6.333...`

---

## ⚖️ 两种方法的对比

| 维度 | 批量平均（本笔记） | 增量更新（图片算法） |
|------|-------------------|---------------------|
| **内存消耗** | O(总样本数) | O(1) |
| **计算复杂度** | O(N) 每次更新 | O(1) 每次更新 |
| **实现难度** | 简单（Python一行） | 中等（需要维护N） |
| **数学等价性** | ✅ 等价 | ✅ 等价 |
| **适合场景** | 教学、调试、小规模问题 | 实际应用、大规模问题 |
| **扩展性** | 不适合在线学习 | 适合在线学习 |

---

## 🎓 为什么本笔记选择批量平均？

### ✅ 教学优势
1. **概念清晰**：直接对应"平均回报"的定义
2. **易于理解**：`np.mean(returns)` 一目了然
3. **便于调试**：可以查看所有历史样本
4. **代码简洁**：无需额外的计数器变量

### 📚 教学流程的优先级
```
概念理解 > 内存效率
直观性 > 计算速度
```

在教学环境中，我们优先让你理解"Q值是回报的平均值"这个核心概念，而不是一开始就引入增量更新的技巧。

---

## 🚀 增量更新的实际应用价值

### 为什么实际应用中更常用增量更新？

#### 1. **内存效率**
```python
# 批量平均：需要存储所有样本
returns = defaultdict(list)  # 可能存储数百万个样本

# 增量更新：只需要O(1)内存
Q = np.zeros((num_states, num_actions))
N = np.zeros((num_states, num_actions))
```

#### 2. **计算效率**
```python
# 批量平均：每次O(N)计算
Q[s, a] = np.mean(returns[(s, a)])  # 遍历所有样本

# 增量更新：每次O(1)计算
Q[s, a] += (1/N[s, a]) * (G - Q[s, a])  # 常数时间
```

#### 3. **在线学习**
- 增量更新可以实时更新，不需要保存历史数据
- 适合永不停止的连续学习场景（如机器人、游戏AI）

---

## 💡 增量更新的代码实现示例

如果你想按照图片中的算法实现，可以这样修改：

```python
def run_mc_epsilon_greedy_incremental(gif_filename='mc_incremental.gif', fps=10):
    """使用增量更新的MC ε-贪心算法"""
    
    # 初始化
    num_states = GRID_SIZE * GRID_SIZE
    num_actions = 4
    
    Q = np.zeros((num_states, num_actions))
    N = np.zeros((num_states, num_actions))  # 🔑 访问次数计数器
    
    # 主循环
    for k in range(1, NUM_EPISODES + 1):
        # 动态调整 ε（如图片算法）
        epsilon = 1.0 / k  # ε衰减
        
        # 创建 ε-贪心策略
        policy = create_epsilon_greedy_policy(Q, epsilon, num_actions)
        
        # 生成回合
        episode = env.generate_episode(policy, MAX_STEPS)
        
        # 计算回报并增量更新 Q 值
        G = 0
        visited_sa_pairs = set()
        
        for state, action, reward in reversed(episode):
            G = reward + GAMMA * G
            sa_pair = (state, action)
            
            # First-visit 检查
            if sa_pair not in visited_sa_pairs:
                N[state, action] += 1  # 🔑 增加访问次数
                
                # 🔑 增量更新公式
                alpha = 1.0 / N[state, action]  # 学习率 = 1/N
                Q[state, action] = Q[state, action] + alpha * (G - Q[state, action])
                
                visited_sa_pairs.add(sa_pair)
    
    return Q

# ✅ 这个版本与图片算法完全一致！
```

### 🔑 关键修改点

1. **添加 N 计数器**：`N = np.zeros((num_states, num_actions))`
2. **增量更新公式**：`Q += (1/N) * (G - Q)`
3. **ε衰减**：`epsilon = 1.0 / k`（可选，图片中有）

---

## 📋 完整对照表：图片算法 vs 本笔记实现

| 图片算法步骤 | 图片伪代码 | 本笔记实现 | 数学等价性 |
|-------------|-----------|-----------|----------|
| 初始化Q | `Q(s,a) ← 0` | `Q = np.zeros(...)` | ✅ |
| 初始化计数器 | `N(s,a) ← 0` | `returns = defaultdict(list)` | ✅（隐式） |
| 生成回合 | 采样回合 | `env.generate_episode()` | ✅ |
| 计算回报 | `G_t ← R_{t+1} + γG_{t+1}` | `G = reward + GAMMA * G` | ✅ |
| 更新访问次数 | `N(s,a) ← N(s,a) + 1` | `len(returns[(s,a)])` | ✅（隐式） |
| 更新Q值 | `Q += (1/N)*(G-Q)` | `Q = mean(returns)` | ✅ 等价！ |
| 策略更新 | `π ← ε-贪心(Q)` | `create_epsilon_greedy_policy(Q)` | ✅ |

---

## 🎯 总结与建议

### ✅ 你现在应该理解的关键点

1. **图片算法用增量更新，本笔记用批量平均**
2. **两种方法数学上完全等价**
3. **批量平均更直观，增量更新更高效**
4. **本笔记优先教学清晰性，实际应用优先效率**

### 🚀 学习路径建议

1. **当前阶段**：使用批量平均（本笔记实现）
   - 理解"Q值 = 回报平均值"这个核心概念
   - 便于调试和可视化

2. **进阶阶段**：学习增量更新
   - 理解增量公式的推导
   - 适应更高效的实现方式

3. **实战阶段**：使用增量更新
   - 处理大规模问题
   - 在线学习场景

### 💡 下一步

- 如果你想实践增量更新，可以复制上面的 `run_mc_epsilon_greedy_incremental()` 函数
- 运行并对比两种实现的结果（应该完全相同！）
- 思考：如果用常数学习率 `α = 0.1` 代替 `1/N`，会有什么影响？

---

⏸️ **理解了Q值更新的两种方法后，你就完全掌握了图片算法与本笔记实现的对应关系！**